# Bokeh的HoverTool工具

In [3]:
import os
os.environ["BOKEH_MINIFIED"] = "false"
from bokeh.io import show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

## 显示鼠标的数据坐标

下面创建一个`HoverTool`工具对象，将其`point_policy`属性设置为鼠标跟随`"follow_mouse"`，将`tooltips`属性设置为显示数据坐标，将`show_arrow`属性设置为`False`关闭箭头显示。并创建一个隐藏的方形图形，将该图形添加进`hover`的`renderers`属性中。

In [11]:
import numpy as np
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)
fig = figure(height=250)
hover = HoverTool(point_policy="follow_mouse",  tooltips=[("xy", "$x, $y")], show_arrow=False)
fig.line(x, y)
q = fig.quad(x[0], x[-1], 1.5, -1.5, alpha=0, visible=True)
hover.renderers = [q]
fig.tools.append(hover)
show(fig)

## 响应HoverTool的回调函数

下面的例子中设置`HoverTool`的`callback`为一个`CustomJS`对象，当鼠标移动到离散点上时会调用其中的回调函数，在下方图表中更新曲线。在回调函数中通过`cb_data`可以获取被选中点的下标。

In [14]:
import numpy as np
from bokeh.io import show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.plotting import figure
from bokeh.layouts import column

source1 = ColumnDataSource(data={"freq":np.random.uniform(2, 10, 10), "amp":np.random.uniform(-2, 2, 10)})
source2 = ColumnDataSource(data={"x":np.linspace(-5, 5, 200), "y":np.zeros(200)})

fig1 = figure(plot_height=200)
fig2 = figure(plot_height=200)
circle = fig1.circle("freq", "amp", source=source1, size=10)
line = fig2.line("x", "y", source=source2)

def callback(source1=source1, source2=source2):
    indices = cb_data.index["1d"].indices
    if len(indices) > 0:
        i = indices[0]
        f = source1.data.freq[i]
        a = source1.data.amp[i]
        for j in range(len(source2.data.x)):
            source2.data.y[j] = a * Math.sin(f * source2.data.x[j])
        source2.change.emit()
        
        
hover = HoverTool(callback=CustomJS.from_py_func(callback))
fig1.add_tools(hover)

show(column(fig1, fig2))